In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

/home/user/projects/domain-specific-llm/venv11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset = load_dataset("ruslanmv/ai-medical-chatbot", split="train[:10%]")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = "<|finetune_right_pad_id|>"                                     
def format_chat_template(row):
    row_json = [
        {"role": "user", "content": row["Patient"]},
        {"role": "assistant", "content": row["Doctor"]},
    ]
    row = tokenizer.apply_chat_template(row_json, tokenize=False)
    tokenized_row = tokenizer(row, padding="max_length", max_length=2048)

    return tokenized_row

tokenized_chat_dataset = dataset.map(
    format_chat_template,
    num_proc=4,
    remove_columns=["Description", "Patient", "Doctor"],
)

Map (num_proc=4):   0%|          | 0/25692 [00:00<?, ? examples/s]

In [ ]:
tokenized_chat_dataset['input_ids'][0]

In [5]:
model_checkpoint = "checkpoint-10500"
save_path = "Llama-3.2-1B-abstract2-v4"
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('Llama-3.2-1B-abstract2-v4/tokenizer_config.json',
 'Llama-3.2-1B-abstract2-v4/special_tokens_map.json',
 'Llama-3.2-1B-abstract2-v4/tokenizer.json')